# Evaluation on DIV2K validation set  (Model using perceptual loss)

Evaluate two metrics PSNR and SSIM of trained models on 100 validation images of bicubic and unknown degradations at scale of 2 and 4.



In [1]:
from tensorflow import config

gpu_devices = config.experimental.list_physical_devices('GPU')

for device in gpu_devices: config.experimental.set_memory_growth(device, True)

## Load testing images from directory

In [3]:
# load testing images from directory

import os
import numpy as np
from PIL import Image

# LR_valid_path = './datasets/DIV2K_valid_LR_unknown/X2/'
# LR_valid_path = './datasets/DIV2K_valid_LR_unknown/X4/'
# LR_valid_path = './datasets/DIV2K_valid_LR_bicubic/X4/'

LR_valid_path = './datasets/DIV2K_valid_LR_bicubic/X2/'
HR_valid_path = './datasets/DIV2K_valid_HR//'

LR_train_imgs = []
HR_train_imgs = []

for path, subpath, files in os.walk(LR_train_path):
    files.sort()
    for i in files:
        if i == '.DS_Store':
            continue
        img = Image.open(LR_train_path + i)
        LR_train_imgs.append(np.asarray(img))

for path, subpath, files in os.walk(HR_train_path):
    files.sort()
    for i in files:
        if i == '.DS_Store':
            continue
        img = Image.open(HR_train_path + i)
        HR_train_imgs.append(np.asarray(img)) 

print(len(LR_train_imgs))
print(len(HR_train_imgs))

100
100


## define perceptual_loss

In [5]:
# define the perceptual_loss so that the X2 and X4 models can be loaded

from keras.applications.vgg19 import VGG19
from keras.layers import Input, Lambda
from keras.models import Model
import keras

def get_VGG19(input_size):
    vgg_inp = Input(input_size)
    vgg = VGG19(include_top=False, input_tensor=vgg_inp)
    for l in vgg.layers: 
        l.trainable = False
    vgg_outp = vgg.get_layer('block2_conv2').output 
    
    return vgg_inp, vgg_outp

def perceptual_loss_x2(y_true, y_pred):
    
    y_t = vgg_content1(y_true)
    y_p = vgg_content1(y_pred)
    loss = keras.losses.mean_squared_error(y_t, y_p)
    
    return loss


def perceptual_loss_x4(y_true, y_pred):
    
    y_t = vgg_content2(y_true)
    y_p = vgg_content2(y_pred)
    loss = keras.losses.mean_squared_error(y_t, y_p)
    
    return loss


vgg_inp, vgg_outp = get_VGG19(input_size=(96,96,3))
vgg_content1 = Model(vgg_inp, vgg_outp)
vgg_inp, vgg_outp = get_VGG19(input_size=(192,192,3))
vgg_content2 = Model(vgg_inp, vgg_outp)
#vgg_content.summary()

Using TensorFlow backend.


## Load trained model for evaluation

In [6]:
import tensorflow as tf
from keras.models import load_model

model = load_model('./model_and_history/final3_perceptual_unknown_4848_12000_subpixel_X2.h5', 
                   custom_objects={'tf': tf, 'perceptual_loss_x2':perceptual_loss_x2})

#model = load_model('./model_and_history/final3_perceptual_unknown_4848_12000_subpixel_X4.h5', 
#                   custom_objects={'tf': tf, 'perceptual_loss_x2':perceptual_loss_x2, 
#                                   'perceptual_loss_x4':perceptual_loss_x4})

#model.summary()

In [7]:
# normaliza imgs from 0~255 to 0~1

def normalize(imgs):
    return imgs / 255

def denormalize(imgs):
    imgs = imgs * 255
    return imgs.astype(np.uint8)

for i in range(len(LR_valid_imgs)):
    #LR_valid_imgs[i] = LR_valid_imgs[i]/255
    LR_valid_imgs[i] = normalize(LR_valid_imgs[i])

## Use trained model to predict test images
Smaller stride can result in better performance but cost more time to recover as more patches are involved. Time needed to predict 100 validation images are recorded with corresponding stride.

In [8]:
# predict and reconstruct test images
# stride should be smaller than patch size to cover all the pixels

from extract_patches import *

test_num = 100
patch_height = 48
patch_width = 48
stride = 10
up_scale = 2

import time

time_start=time.time()

predicted_HR_list = test_patch(LR_valid_imgs, test_num, patch_height, patch_width, stride, model, up_scale)

time_end=time.time()
print('Time cost to predict: ', time_end-time_start, 's')

for i in range(len(predicted_HR_list)):
    predicted_HR_list[i] = denormalize(predicted_HR_list[i])

# perceptual bi x2
# stride=30, test_num=100, Time cost to predict:  280.6533057689667 s

# perceptual bi x4
# stride=30, test_num=100, Time cost to predict:  142.2729160785675 s

# perceptual unknown x2
# stride=30, test_num=100, Time cost to predict:  259.84274101257324 s

# perceptual unknown x4
# stride=30, test_num=100, Time cost to predict:  143.250426530838 s


Time cost to predict:  18.158096313476562 s


In [9]:
# compare with HR images 
# calculate PSNR(peak_signal_noise_ratio) and SSIM(structural_similarity) metrics

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

PSNR_val = []
SSIM_val = []

for i in range(len(predicted_HR_list)):
    PSNR = peak_signal_noise_ratio(HR_valid_imgs[i], predicted_HR_list[i])
    SSIM = structural_similarity(HR_valid_imgs[i], predicted_HR_list[i], multichannel=True)
    PSNR_val.append(PSNR)
    SSIM_val.append(SSIM)

print('PSNR: ', PSNR_val)
print('SSIM: ', SSIM_val)

PSNR:  [28.19379978797455]
SSIM:  [0.5385354790849162]


In [ ]:
# save evaluation results

import pickle

with open('./evaluation/unknown_x4_perceptual_psnr_stride30.pkl','wb') as f2:
    pickle.dump(PSNR_val, f2)

with open('./evaluation/unknown_x4_perceptual_ssim_stride30.pkl','wb') as f3:
    pickle.dump(SSIM_val, f3)

## compare the results

In [1]:
import pickle

with open('./evaluation/bi_x4_perceptual_psnr_stride30.pkl','rb') as f:
    psnr_res = pickle.load(f)
with open('./evaluation/bi_x4_perceptual_ssim_stride30.pkl','rb') as f:
    ssim_res = pickle.load(f)

In [2]:
import numpy as np

print('psnr_max:', max(psnr_res), ', and image index is:', psnr_res.index(max(psnr_res)))
print('psnr_min:', min(psnr_res), ', and image index is:', psnr_res.index(min(psnr_res)))
print('psnr_median:', np.median(psnr_res))
print('psnr_mean:', np.mean(psnr_res))

psnr_max: 32.981269088058646 , and image index is: 42
psnr_min: 16.60940354909153 , and image index is: 94
psnr_median: 21.904707620288484
psnr_mean: 22.01498075258402


In [3]:
print('ssim_max:', max(ssim_res), ', and image index is:', ssim_res.index(max(ssim_res)))
print('ssim_min:', min(ssim_res), ', and image index is:', ssim_res.index(min(ssim_res)))
print('ssim_median:', np.median(ssim_res))
print('ssim_mean:', np.mean(ssim_res))

ssim_max: 0.8970702974687187 , and image index is: 42
ssim_min: 0.32360193413110966 , and image index is: 6
ssim_median: 0.5374477106387275
ssim_mean: 0.5308737822804946
